## To find all of the 'interesting' sources, first remove every island that only has 1 source, then 'flatten' the remaining so they are only represented by a single point in the vo table

### Assumption: the most interesting radio objects have multiple sources per island

In [1]:
import numpy as np
from scipy import stats
import astropy.table 
from astropy.table import Table
import matplotlib.pyplot as plt

In [3]:
t = Table.read(r'C:\Users\clent\Documents\4th Year Project Code\MEERKAT CATALOGS FULL\G288.5-0.5IFx_Mosaic_Mom0_comp.vot')
print(t.colnames)
print(t)

['island', 'source', 'background', 'local_rms', 'lon_str', 'lat_str', 'lon', 'err_lon', 'lat', 'err_lat', 'peak_flux', 'err_peak_flux', 'int_flux', 'err_int_flux', 'a', 'err_a', 'b', 'err_b', 'pa', 'err_pa', 'flags', 'residual_mean', 'residual_std', 'uuid', 'psf_a', 'psf_b', 'psf_pa']
island source   background     local_rms   ...  psf_a   psf_b  psf_pa
------ ------ -------------- ------------- ... ------- ------- ------
     1      0 -0.00038238958  9.391315e-05 ... 7.99992 7.99992    8.0
     2      0 -3.4013727e-05 8.2682396e-05 ... 7.99992 7.99992    8.0
     3      0  0.00012984972 0.00011368903 ... 7.99992 7.99992    8.0
     3      1  3.5077854e-05  9.156917e-05 ... 7.99992 7.99992    8.0
     5      0  9.1852875e-05 0.00013887978 ... 7.99992 7.99992    8.0
     6      0  0.00017014833 0.00012365321 ... 7.99992 7.99992    8.0
     7      0  -0.0009232825 0.00010227811 ... 7.99992 7.99992    8.0
     8      0  0.00017107005 0.00011957841 ... 7.99992 7.99992    8.0
     9      0 

In [4]:
mult_sources = t[t['source'] == 1]
print(mult_sources)
mult_sources.write(r'C:\Users\clent\Documents\4th Year Project Code\MeerKAT Catalogs\G288.5-0.5_MULT_SOURCES.vot', overwrite=True, format='votable')

island source   background     local_rms   ...  psf_a   psf_b  psf_pa
------ ------ -------------- ------------- ... ------- ------- ------
     3      1  3.5077854e-05  9.156917e-05 ... 7.99992 7.99992    8.0
    18      1  0.00047630217 8.6980945e-05 ... 7.99992 7.99992    8.0
    24      1   -0.000211079 0.00012891782 ... 7.99992 7.99992    8.0
    56      1  0.00024852922 0.00013152474 ... 7.99992 7.99992    8.0
   174      1  0.00045054225  8.786525e-05 ... 7.99992 7.99992    8.0
   212      1   0.0005668045 5.4815904e-05 ... 7.99992 7.99992    8.0
   304      1 -0.00033638888 8.1574224e-05 ... 7.99992 7.99992    8.0
   305      1   0.0005219377  4.880799e-05 ... 7.99992 7.99992    8.0
   307      1  0.00016515676 2.9500678e-05 ... 7.99992 7.99992    8.0
   346      1  0.00022079711  8.376038e-05 ... 7.99992 7.99992    8.0
   ...    ...            ...           ... ...     ...     ...    ...
  3941      1 -1.1715517e-06 1.8615925e-05 ... 7.99992 7.99992    8.0
  3953      1 -5.035

## Found 'objects of interest' - next step would be to maintain all of the sources within these multi-source islands and find the mean lat and lon (i.e. the centroid), which would then allow a cut to be made around the point for the purposes of producing a training set

In [6]:
newT = Table.read(r'C:\Users\clent\Documents\4th Year Project Code\MEERKAT CATALOGS FULL\G288.5-0.5IFx_Mosaic_Mom0_comp.vot')
print(newT.colnames)
print(newT)
# newT.show_in_browser(jsviewer=True)

['island', 'source', 'background', 'local_rms', 'lon_str', 'lat_str', 'lon', 'err_lon', 'lat', 'err_lat', 'peak_flux', 'err_peak_flux', 'int_flux', 'err_int_flux', 'a', 'err_a', 'b', 'err_b', 'pa', 'err_pa', 'flags', 'residual_mean', 'residual_std', 'uuid', 'psf_a', 'psf_b', 'psf_pa']
island source   background     local_rms   ...  psf_a   psf_b  psf_pa
------ ------ -------------- ------------- ... ------- ------- ------
     1      0 -0.00038238958  9.391315e-05 ... 7.99992 7.99992    8.0
     2      0 -3.4013727e-05 8.2682396e-05 ... 7.99992 7.99992    8.0
     3      0  0.00012984972 0.00011368903 ... 7.99992 7.99992    8.0
     3      1  3.5077854e-05  9.156917e-05 ... 7.99992 7.99992    8.0
     5      0  9.1852875e-05 0.00013887978 ... 7.99992 7.99992    8.0
     6      0  0.00017014833 0.00012365321 ... 7.99992 7.99992    8.0
     7      0  -0.0009232825 0.00010227811 ... 7.99992 7.99992    8.0
     8      0  0.00017107005 0.00011957841 ... 7.99992 7.99992    8.0
     9      0 

In [7]:
newT_single = astropy.table.unique(newT, keys='island', keep='none')
print(newT_single)
newT_combined = astropy.table.vstack(([newT, newT_single]))
print(newT_combined)
newT_mult = astropy.table.unique(newT_combined, keep='none', silent=True)
print(newT_mult)
newT_mult.show_in_browser(jsviewer=True)
newT_mult.write(r'C:\Users\clent\Documents\4th Year Project Code\MeerKAT Catalogs\G288.5-0.5_big_sources.vot', overwrite=True, format='votable')

island source   background     local_rms    ...  psf_a   psf_b  psf_pa
------ ------ -------------- -------------- ... ------- ------- ------
     1      0 -0.00038238958   9.391315e-05 ... 7.99992 7.99992    8.0
     2      0 -3.4013727e-05  8.2682396e-05 ... 7.99992 7.99992    8.0
     5      0  9.1852875e-05  0.00013887978 ... 7.99992 7.99992    8.0
     6      0  0.00017014833  0.00012365321 ... 7.99992 7.99992    8.0
     7      0  -0.0009232825  0.00010227811 ... 7.99992 7.99992    8.0
     8      0  0.00017107005  0.00011957841 ... 7.99992 7.99992    8.0
     9      0 -3.5001558e-05   7.432467e-05 ... 7.99992 7.99992    8.0
    10      0  -0.0006618347   0.0001219761 ... 7.99992 7.99992    8.0
    12      0  0.00040274934 0.000107009975 ... 7.99992 7.99992    8.0
    13      0   5.032751e-05  9.0600486e-05 ... 7.99992 7.99992    8.0
   ...    ...            ...            ... ...     ...     ...    ...
  4355      0   0.0001653752  4.4181066e-05 ... 7.99992 7.99992    8.0
  4357

By applying the $\texttt{unique}$ function twice, we can first remove all of the desired objects, and then after a combining the new table with the original, remove all of the undesired objects

In [9]:
newT_by_island = newT_mult.group_by('island')
finalT = newT_by_island.groups.aggregate(np.mean)
print(finalT)
finalT = Table([finalT['island'], finalT['lon'], finalT['lat']])
print(finalT)
finalT.write(r'C:\Users\clent\Documents\4th Year Project Code\MeerKAT Catalogs\G288.5-0.5_FINAL.vot', overwrite=True, format='votable')

island source        background       ...  psf_a   psf_b  psf_pa
------ ------ ----------------------- ... ------- ------- ------
     3    0.5   8.246379002230242e-05 ... 7.99992 7.99992    8.0
    18    0.5   0.0004749960789922625 ... 7.99992 7.99992    8.0
    24    0.5 -0.00020865077385678887 ... 7.99992 7.99992    8.0
    56    0.5  0.00023400728241540492 ... 7.99992 7.99992    8.0
   174    0.5   0.0004473334411159158 ... 7.99992 7.99992    8.0
   212    0.5    0.000569575815461576 ... 7.99992 7.99992    8.0
   304    0.5  -0.0003370051272213459 ... 7.99992 7.99992    8.0
   305    1.0   0.0005230777896940708 ... 7.99992 7.99992    8.0
   307    0.5  0.00016532308654859662 ... 7.99992 7.99992    8.0
   346    0.5   0.0002197534777224064 ... 7.99992 7.99992    8.0
   ...    ...                     ... ...     ...     ...    ...
  3941    0.5  -1.985719109143247e-07 ... 7.99992 7.99992    8.0
  3953    0.5 -5.1295748562552035e-05 ... 7.99992 7.99992    8.0
  3991    0.5   9.1064757